<a href="https://colab.research.google.com/github/Nayoung-young/kaggle_practice/blob/main/Clothes_Size_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clothes-size-prediction 


*   출처: https://www.kaggle.com/tourist55/clothessizeprediction 



## 1. 데이터 전처리 
(8/10)

* 파일 업로드, 데이터 확인 
* data -> size_input, size_target으로 분리


In [ ]:
from google.colab import files 
myfile = files.upload()

Saving clothes-size-prediction.csv to clothes-size-prediction (1).csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import io 
import pandas as pd 

In [ ]:
data = pd.read_csv(io.BytesIO(myfile['clothes-size-prediction.csv']))

In [ ]:
data.head()

,weight,age,height,size
0,62,28.0,172.72,XL
1,59,36.0,167.64,L
2,61,34.0,165.10,M
3,65,27.0,175.26,L
4,62,45.0,172.72,M


In [ ]:
data.describe()

,weight,age,height
count,119734.000000,119477.000000,119404.000000
mean,61.756811,34.027311,165.805794
std,9.944863,8.149447,6.737651
min,22.000000,0.000000,137.160000
25%,55.000000,29.000000,160.020000
50%,61.000000,32.000000,165.100000
75%,67.000000,37.000000,170.180000
max,136.000000,117.000000,193.040000


In [ ]:
print(data.shape)

(119734, 4)


In [ ]:
print(pd.unique(data['size']))
# size 몇 가지 있는지 확인

['XL' 'L' 'M' 'S' 'XXS' 'XXXL' 'XXL']


In [ ]:
import matplotlib.pyplot as plt
weight_arr = data.weight
age_arr = data.age
height_arr = data.height

'''plt.scatter(np.arange(), weight_arr)
plt.scatter(np.arange(200), age_arr)
plt.scatter(np.arange(200), height_arr)'''

weight_arr.size


119734

**훈련 세트와 테스트 세트 표준화 전처리**

In [ ]:
# size를 타깃 열로 설정, 나머지 3개 열은 입력 데이터로 사용 
import numpy as np
size_input = data[['weight', 'age', 'height']].to_numpy()

In [ ]:
print(size_input[:5])

[[ 62.    28.   172.72]
 [ 59.    36.   167.64]
 [ 61.    34.   165.1 ]
 [ 65.    27.   175.26]
 [ 62.    45.   172.72]]


In [ ]:
size_target = data[['size']].to_numpy()
print(size_target[:5])

[['XL']
 ['L']
 ['M']
 ['L']
 ['M']]


In [ ]:
from sklearn.model_selection import train_test_split 
train_input, test_input, train_target, test_target = train_test_split(
    size_input, size_target, test_size=0.2
)

In [ ]:
print(train_input.shape, test_input.shape)
print(train_input[:5])

(95787, 3) (23947, 3)
[[ 63.    27.   167.64]
 [ 65.    26.   172.72]
 [ 56.    30.   160.02]
 [ 47.    29.   157.48]
 [ 71.    36.   170.18]]


In [ ]:
# 훈련 세트 전처리
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

## 2. 여러 가지 모델 사용해보기 

* 로지스틱 회귀 
* 결정 트리 
* 앙상블